In [4]:
import cv2 
import matplotlib.pyplot as plt
import numpy as np

In [5]:
imgs = []
img_path = ['Leon1.jpg', 'Leon2.jpg']


for i in range(len(img_path)):
    imgs.append(cv2.imread(img_path[i]))
    imgs[i]=cv2.resize(imgs[i],(0,0),fx=0.4,fy=0.4) 


stitching = cv2.Stitcher.create()
(dummy,output)=stitching.stitch(imgs) 

# output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

if dummy != cv2.STITCHER_OK:
    print("Not success")
else:
    print("Success")



# plt.imshow(output)
# plt.show()

Not success
None


In [6]:
def detect_and_match_features(img1, img2):
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(img1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(img2, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

    return keypoints1, keypoints2, matches

keypoints1, keypoints2, matches = detect_and_match_features(imgs[0], imgs[1])

In [7]:
def estimate_homography(keypoints1, keypoints2, matches, threshold=3):
    src_points = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_points = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    H, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, threshold)
    return H, mask

H, mask = estimate_homography(keypoints1, keypoints2, matches)

In [8]:
def warp_images(img1, img2, H):
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    corners1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)
    corners2 = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)
    warped_corners2 = cv2.perspectiveTransform(corners2, H)

    corners = np.concatenate((corners1, warped_corners2), axis=0)
    [xmin, ymin] = np.int32(corners.min(axis=0).ravel() - 0.5)
    [xmax, ymax] = np.int32(corners.max(axis=0).ravel() + 0.5)

    t = [-xmin, -ymin]
    Ht = np.array([[1, 0, t[0]], [0, 1, t[1]], [0, 0, 1]])

    warped_img2 = cv2.warpPerspective(img2, Ht @ H, (xmax - xmin, ymax - ymin))
    warped_img2[t[1]:h1 + t[1], t[0]:w1 + t[0]] = img1

    return warped_img2

warped_img = warp_images(imgs[0], imgs[1], H)

In [9]:
def blend_images(img1, img2):
    mask = np.where(img1 != 0, 1, 0).astype(np.float32)
    blended_img = img1 * mask + img2 * (1 - mask)
    return blended_img.astype(np.uint8)

output_img = blend_images(warped_img, imgs[1])

ValueError: operands could not be broadcast together with shapes (284,226,3) (276,234,3) 

In [ ]:
plt.imshow(output_img)
plt.show()